In [5]:
from ultralytics import YOLO
model = YOLO("modelFolder/model_v7.pt")
results = model(source="validimage/5_Color.png",device="cpu", project = "results",name="test6d", conf=0.8 )
for result in results:
    image_points = result.masks.xyn
    print(len(image_points))
    # xy = result.masks.xy
    # print(image_points)
    # print(xy)
    


image 1/1 c:\Team_Project-1\validimage\5_Color.png: 384x640 1 box, 900.1ms
Speed: 0.0ms preprocess, 900.1ms inference, 11.6ms postprocess per image at shape (1, 3, 384, 640)


1


In [53]:
from crop_ply import *
xyz_output= cropPly(image_points,"validimage/5.ply")
print(xyz_output)

[PointCloud with 3588 points.]


In [4]:
import cv2
import pyrealsense2 as rs
import numpy as np
from ultralytics import YOLO

# RealSense 카메라 스트리밍 설정
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 360, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 360, rs.format.z16, 30)
pipeline.start(config)

# RealSense 정렬 프로세서 생성
align_to = rs.stream.color
align = rs.align(align_to)

# YOLO 모델 로드 (예: Darknet 또는 다른 라이브러리 사용)
model = YOLO("modelFolder/model_v7.pt")

# 클래스별 고유한 색상 정의
class_colors = {
    0: (0, 255, 0),   # 클래스 0의 색상 (예: 녹색)
    1: (255, 0, 0),   # 클래스 1의 색상 (예: 빨강)
    2: (0, 0, 255),   # 클래스 2의 색상 (예: 파랑)
    3: (255, 255, 0), # 클래스 3의 색상 (예: 노랑)
    4: (128, 0, 128), # 클래스 4의 색상 (예: 보라)
}

while True:
    # RealSense 카메라 프레임 가져오기
    frames = pipeline.wait_for_frames()
    
    # 컬러 이미지와 깊이 이미지 정렬
    aligned_frames = align.process(frames)
    aligned_color_frame = aligned_frames.get_color_frame()
    aligned_depth_frame = aligned_frames.get_depth_frame()

    if not aligned_color_frame or not aligned_depth_frame:
        continue

    # 정렬된 컬러 이미지와 깊이 이미지를 OpenCV 이미지로 변환
    color_image = np.asanyarray(aligned_color_frame.get_data())
    depth_image = np.asanyarray(aligned_depth_frame.get_data())

    # YOLO를 사용하여 객체 검출 및 세그멘테이션 수행
    results = model(color_image, conf=0.8, device="cpu")
    
    # 결과를 color_image와 depth_image에 오버레이
    for result in results:
        # print("mask",result.masks)
        for box, cls, conf in zip(result.boxes.xywh.tolist(), result.boxes.cls.tolist(), result.boxes.conf.tolist()):
            x, y, w, h = box
            x1, y1, x2, y2 = int(x - w / 2), int(y - h / 2), int(x + w / 2), int(y + h / 2)
            label = f"{model.names[int(cls)]}: {conf:.2f}"
            color_image = cv2.rectangle(color_image, (x1, y1), (x2, y2), class_colors[cls], 2)
            color_image = cv2.putText(color_image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            depth_image = cv2.rectangle(depth_image, (x1, y1), (x2, y2), class_colors[cls], 2)

    # 컬러 이미지를 따로 표시
    cv2.imshow("Color Image", color_image)

    # 깊이 이미지 시각화
    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.1), cv2.COLORMAP_JET)
    cv2.imshow("Depth Image", depth_colormap)

    # 'q' 키를 눌러 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 정리 및 종료
pipeline.stop()
cv2.destroyAllWindows()



0: 384x640 1 box, 915.1ms
Speed: 8.1ms preprocess, 915.1ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 box, 942.5ms
Speed: 3.0ms preprocess, 942.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 box, 913.1ms
Speed: 0.0ms preprocess, 913.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 box, 872.3ms
Speed: 0.0ms preprocess, 872.3ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 box, 882.4ms
Speed: 2.0ms preprocess, 882.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 box, 903.0ms
Speed: 0.0ms preprocess, 903.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 box, 847.3ms
Speed: 1.0ms preprocess, 847.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 box, 862.4ms
Speed: 2.0ms preprocess, 862.4ms inference, 8.1ms postprocess per image at shape (1, 3, 384, 640)


In [19]:
import open3d as o3d
import numpy as np


# 포인트 클라우드를 시각화
for cloud in xyz_output:
    # 포인트 클라우드의 경계 상자 계산
    aabb = cloud.get_axis_aligned_bounding_box()
    aabb.color = (1, 0, 0)

    obb = cloud.get_oriented_bounding_box()
    obb.color = (0, 1, 0)

    # 클라우드 포인트의 중심 위치 계산
    center = np.mean(np.asarray(cloud.points), axis=0)
    print(center)

    # 중앙 위치에 좌표 프레임 메시 생성
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.07)  # size를 조절하여 좌표 프레임 크기 설정 가능

    # 중앙 위치에 좌표 프레임 메시를 이동
    mesh.translate(center)

    # 시각화
    o3d.visualization.draw_geometries([cloud, aabb, obb, mesh])

    # PCA를 수행하여 주요 주성분 벡터를 찾습니다.
    points = np.asarray(cloud.points)
    covariance_matrix = np.cov(points, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

    # 주요 주성분 벡터와 각각의 축과의 각도를 계산합니다.
    x_axis = np.array([1, 0, 0])  # x 축
    y_axis = np.array([0, 1, 0])  # y 축
    z_axis = np.array([0, 0, 1])  # z 축

    angle_x_rad = np.arccos(np.dot(eigenvectors[:, -1], x_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(x_axis)))
    angle_x_deg = np.degrees(angle_x_rad)

    angle_y_rad = np.arccos(np.dot(eigenvectors[:, -1], y_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(y_axis)))
    angle_y_deg = np.degrees(angle_y_rad)

    angle_z_rad = np.arccos(np.dot(eigenvectors[:, -1], z_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(z_axis)))
    angle_z_deg = np.degrees(angle_z_rad)

    
    print(f"(x 축 기준): {angle_x_deg}도")
    print(f"(y 축 기준): {angle_y_deg}도")
    print(f"(z 축 기준): {angle_z_deg}도")
    
    o3d.io.write_point_cloud("robot/test.ply", cloud)


[  -0.039454   -0.023009    -0.64544]
(x 축 기준): 142.94969940274632도
(y 축 기준): 127.03310642746156도
(z 축 기준): 90.97338359234107도


In [114]:
import open3d as o3d
import numpy as np
from ultralytics import YOLO
from crop_ply import *

model = YOLO("modelFolder/model_v7.pt")
results = model(source="validimage/3_Color.png",device="cpu", project = "results",name="test6d", conf=0.8 )
for result in results:
    image_points = result.masks.xyn
    print(len(image_points))



xyz_output= cropPly(image_points,"validimage/3.ply")
print(xyz_output)


# 포인트 클라우드를 시각화
for cloud in xyz_output:
    # 포인트 클라우드의 경계 상자 계산
    aabb = cloud.get_axis_aligned_bounding_box()
    aabb.color = (1, 0, 0)
    
    obb = cloud.get_oriented_bounding_box()
    obb.color = (0, 1, 0)

    # aabb의 중심 위치 계산
    center = np.mean(np.asarray(cloud.points), axis=0)

    # # 중앙 위치에 좌표 프레임 메시 생성
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.01)  # size를 조절하여 좌표 프레임 크기 설정 가능
    mesh.translate(center)

    # aabb의 중점을 지나는 X, Y, Z 축 선을 시각화
    aabb_x_axis = o3d.geometry.LineSet()
    aabb_x_axis.points = o3d.utility.Vector3dVector([center, [center[0] + 0.1, center[1], center[2]]])
    aabb_x_axis.lines = o3d.utility.Vector2iVector([[0, 1]])
    aabb_x_axis.colors = o3d.utility.Vector3dVector([[0, 0, 0]])

    aabb_y_axis = o3d.geometry.LineSet()
    aabb_y_axis.points = o3d.utility.Vector3dVector([center, [center[0], center[1] + 0.1, center[2]]])
    aabb_y_axis.lines = o3d.utility.Vector2iVector([[0, 1]])
    aabb_y_axis.colors = o3d.utility.Vector3dVector([[0, 0, 0]])

    aabb_z_axis = o3d.geometry.LineSet()
    aabb_z_axis.points = o3d.utility.Vector3dVector([center, [center[0], center[1], center[2] + 0.1]])
    aabb_z_axis.lines = o3d.utility.Vector2iVector([[0, 1]])
    aabb_z_axis.colors = o3d.utility.Vector3dVector([[0, 0, 0]])

    # obb의 방향 벡터를 사용하여 X, Y, Z 축 선을 시각화
    obb_x_axis = o3d.geometry.LineSet()
    obb_x_axis.points = o3d.utility.Vector3dVector([center, center + 0.1 * obb.R[0, :3]])  # 첫 번째 방향 벡터는 X 축
    obb_x_axis.lines = o3d.utility.Vector2iVector([[0, 1]])
    obb_x_axis.colors = o3d.utility.Vector3dVector([[1, 0, 0]])

    obb_y_axis = o3d.geometry.LineSet()
    obb_y_axis.points = o3d.utility.Vector3dVector([center, center + 0.1 * obb.R[1, :3]])  # 두 번째 방향 벡터는 Y 축
    obb_y_axis.lines = o3d.utility.Vector2iVector([[0, 1]])
    obb_y_axis.colors = o3d.utility.Vector3dVector([[0, 1, 0]])

    obb_z_axis = o3d.geometry.LineSet()
    obb_z_axis.points = o3d.utility.Vector3dVector([center, center + 0.1 * obb.R[2, :3]])  # 세 번째 방향 벡터는 Z 축
    obb_z_axis.lines = o3d.utility.Vector2iVector([[0, 1]])
    obb_z_axis.colors = o3d.utility.Vector3dVector([[0, 0, 1]])

    # # OBB 박스의 방향 벡터 추출
    obb_rotation_matrix = obb.R
    obb_x_axis1 = obb_rotation_matrix[:, 0]  # X 축 방향 벡터
    obb_y_axis1 = obb_rotation_matrix[:, 1]  # Y 축 방향 벡터
    obb_z_axis1 = obb_rotation_matrix[:, 2]  # Z 축 방향 벡터

    # AABB 박스의 각 축 벡터
    aabb_x_axis1 = np.array([1.0, 0.0, 0.0])  # AABB X 축
    aabb_y_axis1 = np.array([0.0, 1.0, 0.0])  # AABB Y 축
    aabb_z_axis1 = np.array([0.0, 0.0, 1.0])  # AABB Z 축

    # 두 박스 간의 각도 계산
    angle_x = np.arccos(np.dot(obb_x_axis1, aabb_x_axis1))
    angle_y = np.arccos(np.dot(obb_y_axis1, aabb_y_axis1))
    angle_z = np.arccos(np.dot(obb_z_axis1, aabb_z_axis1))

    # 라디안을 도(degree)로 변환
    angle_x_deg = np.degrees(angle_x)
    angle_y_deg = np.degrees(angle_y)
    angle_z_deg = np.degrees(angle_z)

    print("Angle between OBB X axis and AABB X axis:", angle_x_deg)
    print("Angle between OBB Y axis and AABB Y axis:", angle_y_deg)
    print("Angle between OBB Z axis and AABB Z axis:", angle_z_deg)
    
    # print("roll:",angle_z_deg)
    # print("pitch:",angle_y_deg)
    # print("yaw:",angle_x_deg)

    # 시각화
    o3d.visualization.draw_geometries([cloud, aabb, obb,aabb_x_axis, aabb_y_axis, aabb_z_axis, obb_x_axis, obb_y_axis, obb_z_axis])



image 1/1 c:\Team_Project-1\validimage\3_Color.png: 384x640 1 box, 932.0ms
Speed: 0.0ms preprocess, 932.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


1
[PointCloud with 1508 points.]
Angle between OBB X axis and AABB X axis: 86.63452049817697
Angle between OBB Y axis and AABB Y axis: 86.62798397036205
Angle between OBB Z axis and AABB Z axis: 3.745617533598035


In [8]:
import open3d as o3d
import numpy as np
from ultralytics import YOLO
from crop_ply import *

model = YOLO("modelFolder/model_v2.pt")
results = model(source="validimage/3_Color.png",device="cpu", project = "results",name="test6d", conf=0.8 )
for result in results:
    image_points = result.masks.xyn
    print(len(image_points))



xyz_output= cropPly(image_points,"validimage/3.ply")
print(xyz_output)


# 포인트 클라우드를 시각화
for cloud in xyz_output:
    # 포인트 클라우드의 경계 상자 계산
    aabb = cloud.get_axis_aligned_bounding_box()
    aabb.color = (1, 0, 0)
    
    obb = cloud.get_oriented_bounding_box()
    obb.color = (0, 1, 0)

    # aabb의 중심 위치 계산
    center = np.mean(np.asarray(cloud.points), axis=0)

    # # 중앙 위치에 좌표 프레임 메시 생성
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.01)  # size를 조절하여 좌표 프레임 크기 설정 가능
    mesh.translate(center)

    # aabb의 중점을 지나는 X, Y, Z 축 선을 시각화
    aabb_x_axis = o3d.geometry.LineSet()
    aabb_x_axis.points = o3d.utility.Vector3dVector([center, [center[0] + 0.1, center[1], center[2]]])
    aabb_x_axis.lines = o3d.utility.Vector2iVector([[0, 1]])
    aabb_x_axis.colors = o3d.utility.Vector3dVector([[0, 0, 0]])

    aabb_y_axis = o3d.geometry.LineSet()
    aabb_y_axis.points = o3d.utility.Vector3dVector([center, [center[0], center[1] + 0.1, center[2]]])
    aabb_y_axis.lines = o3d.utility.Vector2iVector([[0, 1]])
    aabb_y_axis.colors = o3d.utility.Vector3dVector([[0, 0, 0]])

    aabb_z_axis = o3d.geometry.LineSet()
    aabb_z_axis.points = o3d.utility.Vector3dVector([center, [center[0], center[1], center[2] + 0.1]])
    aabb_z_axis.lines = o3d.utility.Vector2iVector([[0, 1]])
    aabb_z_axis.colors = o3d.utility.Vector3dVector([[0, 0, 0]])

    # obb의 방향 벡터를 사용하여 X, Y, Z 축 선을 시각화
    obb_x_axis = o3d.geometry.LineSet()
    obb_x_axis.points = o3d.utility.Vector3dVector([center, center + 0.1 * obb.R[0, :3]])  # 첫 번째 방향 벡터는 X 축
    obb_x_axis.lines = o3d.utility.Vector2iVector([[0, 1]])
    obb_x_axis.colors = o3d.utility.Vector3dVector([[1, 0, 0]])

    obb_y_axis = o3d.geometry.LineSet()
    obb_y_axis.points = o3d.utility.Vector3dVector([center, center + 0.1 * obb.R[1, :3]])  # 두 번째 방향 벡터는 Y 축
    obb_y_axis.lines = o3d.utility.Vector2iVector([[0, 1]])
    obb_y_axis.colors = o3d.utility.Vector3dVector([[0, 1, 0]])

    obb_z_axis = o3d.geometry.LineSet()
    obb_z_axis.points = o3d.utility.Vector3dVector([center, center + 0.1 * obb.R[2, :3]])  # 세 번째 방향 벡터는 Z 축
    obb_z_axis.lines = o3d.utility.Vector2iVector([[0, 1]])
    obb_z_axis.colors = o3d.utility.Vector3dVector([[0, 0, 1]])

    # # OBB 박스의 방향 벡터 추출
    obb_rotation_matrix = obb.R
    obb_x_axis1 = obb_rotation_matrix[:, 0]  # X 축 방향 벡터
    obb_y_axis1 = obb_rotation_matrix[:, 1]  # Y 축 방향 벡터
    obb_z_axis1 = obb_rotation_matrix[:, 2]  # Z 축 방향 벡터

    # AABB 박스의 각 축 벡터
    aabb_x_axis1 = np.array([1.0, 0.0, 0.0])  # AABB X 축
    aabb_y_axis1 = np.array([0.0, 1.0, 0.0])  # AABB Y 축
    aabb_z_axis1 = np.array([0.0, 0.0, 1.0])  # AABB Z 축

    # 두 박스 간의 각도 계산
    angle_x = np.arccos(np.dot(obb_x_axis1, aabb_x_axis1))
    angle_y = np.arccos(np.dot(obb_y_axis1, aabb_y_axis1))
    angle_z = np.arccos(np.dot(obb_z_axis1, aabb_z_axis1))

    # 라디안을 도(degree)로 변환
    angle_x_deg = np.degrees(angle_x)
    angle_y_deg = np.degrees(angle_y)
    angle_z_deg = np.degrees(angle_z)

    print("Angle between OBB X axis and AABB X axis:", angle_x_deg)
    print("Angle between OBB Y axis and AABB Y axis:", angle_y_deg)
    print("Angle between OBB Z axis and AABB Z axis:", angle_z_deg)
    
    # print("roll:",angle_z_deg)
    # print("pitch:",angle_y_deg)
    # print("yaw:",angle_x_deg)

    # 시각화
    o3d.visualization.draw_geometries([cloud, aabb, obb,aabb_x_axis, aabb_y_axis, aabb_z_axis, obb_x_axis, obb_y_axis, obb_z_axis])



image 1/1 c:\Team_Project\validimage\3_Color.png: 384x640 1 box, 657.2ms
Speed: 2.0ms preprocess, 657.2ms inference, 10.0ms postprocess per image at shape (1, 3, 384, 640)


1
[PointCloud with 1487 points.]
Angle between OBB X axis and AABB X axis: 92.2756485068401
Angle between OBB Y axis and AABB Y axis: 87.72705605768478
Angle between OBB Z axis and AABB Z axis: 176.74371018848473


In [204]:
import open3d as o3d
import numpy as np
from ultralytics import YOLO
from crop_ply import *

model = YOLO("modelFolder/model_v7.pt")
results = model(source="validimage/3_Color.png",device="cpu", project = "results",name="test6d", conf=0.8 )
for result in results:
    image_points = result.masks.xyn
    print(len(image_points))



xyz_output= cropPly(image_points,"validimage/3.ply")
print(xyz_output)


# 포인트 클라우드를 시각화
for cloud in xyz_output:
    # 포인트 클라우드의 경계 상자 계산
    aabb = cloud.get_axis_aligned_bounding_box()
    aabb.color = (1, 0, 0)
    
    obb = cloud.get_oriented_bounding_box()
    obb.color = (0, 1, 0)

    # aabb의 중심 위치 계산
    center = np.mean(np.asarray(cloud.points), axis=0)

 

    # # OBB 박스의 방향 벡터 추출
    obb_rotation_matrix = obb.R
    obb_x_axis1 = obb_rotation_matrix[:, 0]  # X 축 방향 벡터
    obb_y_axis1 = obb_rotation_matrix[:, 1]  # Y 축 방향 벡터
    obb_z_axis1 = obb_rotation_matrix[:, 2]  # Z 축 방향 벡터

    # AABB 박스의 각 축 벡터
    aabb_x_axis1 = np.array([1.0, 0.0, 0.0])  # AABB X 축
    aabb_y_axis1 = np.array([0.0, 1.0, 0.0])  # AABB Y 축
    aabb_z_axis1 = np.array([0.0, 0.0, 1.0])  # AABB Z 축

    # 두 박스 간의 각도 계산
    angle_x = np.arccos(np.dot(obb_x_axis1, aabb_x_axis1))
    angle_y = np.arccos(np.dot(obb_z_axis1, aabb_y_axis1))
    angle_z = np.arccos(np.dot(obb_z_axis1, aabb_x_axis1))

    # 라디안을 도(degree)로 변환
    angle_x_deg = np.degrees(angle_x)
    angle_y_deg = np.degrees(angle_y)
    angle_z_deg = np.degrees(angle_z)
    
    print("roll:",90 - angle_y_deg)
    print("pitch:",90 - angle_z_deg)
    print("yaw:", angle_x_deg)

    # 시각화
    # o3d.visualization.draw_geometries([cloud,aabb,obb])



image 1/1 c:\Team_Project-1\validimage\3_Color.png: 384x640 1 box, 882.8ms
Speed: 0.0ms preprocess, 882.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


1
[PointCloud with 1508 points.]
roll: 0.010325887184094995
pitch: -3.7456032597785196
yaw: 86.63452049817697


In [65]:
import open3d as o3d
import numpy as np
from ultralytics import YOLO
from crop_ply import *

# YOLO 모델 초기화
model = YOLO("modelFolder/model_v7.pt")

# 이미지에서 객체 탐지
results = model(source="validimage/3_Color.png", device="cpu", project="result", name="proof", conf=0.8)
for result in results:
    image_points = result.masks.xyn
    print(len(image_points))

# 포인트 클라우드 생성
xyz_output = cropPly(image_points, "validimage/3.ply")
print(xyz_output)

# OBB의 8개 꼭짓점을 추출하는 함수
def get_obb_corners(obb):
    obb_mat = obb.R  # 회전 행렬을 가져옵니다.
    obb_center = obb.center  # 중심 위치를 가져옵니다.
    obb_extents = obb.extent  # 각 축별 크기를 가져옵니다.
    obb_corners = []

    for i in range(8):
        corner = obb_center + obb_mat.dot(obb_extents * (np.array([i & 1, (i & 2) >> 1, (i & 4) >> 2]) * 2 - 1))
        obb_corners.append(corner)

    return obb_corners

# 롤 (Roll), 피치 (Pitch), 요 (Yaw) 각도 계산 함수
def calculate_rotation_angles(obb):
    # 초기 회전 행렬 (단위 행렬) 생성
    initial_rotation_matrix = np.identity(3)

    # 현재 회전 행렬 가져오기
    current_rotation_matrix = obb.R
    
    # 초기 방향과 현재 방향 사이의 차이 계산
    rotation_difference_matrix = current_rotation_matrix @ np.linalg.inv(initial_rotation_matrix)
    
    # 차이 행렬을 Euler 각도로 변환
    roll_rad = np.arctan2(rotation_difference_matrix[2, 1], rotation_difference_matrix[2, 2])
    pitch_rad = np.arctan2(-rotation_difference_matrix[2, 0], np.sqrt(rotation_difference_matrix[2, 1]**2 + rotation_difference_matrix[2, 2]**2))
    yaw_rad = np.arctan2(rotation_difference_matrix[1, 0], rotation_difference_matrix[0, 0])
    
    # 라디안 각도를 도(degree)로 변환
    roll_deg = np.degrees(roll_rad)
    pitch_deg = np.degrees(pitch_rad)
    yaw_deg = np.degrees(yaw_rad)

    return roll_deg, pitch_deg, yaw_deg

# 포인트 클라우드를 시각화하고 OBB의 꼭짓점 및 회전 각도를 출력
for cloud in xyz_output:
    obb = cloud.get_oriented_bounding_box()
    obb.color = (0, 1, 0)

    # # OBB의 8개 꼭짓점 추출
    # obb_corners = get_obb_corners(obb)
    # print("OBB 꼭짓점 좌표:")
    # print(obb_corners)
    print(obb.get_center().tolist())

    # 롤 (Roll), 피치 (Pitch), 요 (Yaw) 각도 계산
    roll_deg, pitch_deg, yaw_deg = calculate_rotation_angles(obb)
    print("롤 (도):", roll_deg)
    print("피치 (도):", pitch_deg)
    print("요 (도):", yaw_deg)



image 1/1 c:\Team_Project\validimage\3_Color.png: 384x640 1 box, 3537.5ms
Speed: 8.0ms preprocess, 3537.5ms inference, 16.0ms postprocess per image at shape (1, 3, 384, 640)


1
[PointCloud with 1508 points.]
[-0.04059727944854639, -0.02656025753912536, -0.6557093019406488]
롤 (도): -3.7397417092000556
피치 (도): -0.2098695370821594
요 (도): 86.63449789482713


In [61]:
import open3d as o3d
import numpy as np

# 직육면체를 생성합니다.
cube = o3d.geometry.TriangleMesh.create_box(width=4.0, height=2.0, depth=1.0)
cube.compute_vertex_normals()

# 회전 각도 설정 (롤, 피치, 요)
roll_angle = np.radians(-8.524368871014845)  # 롤 각도 (30도)
pitch_angle = np.radians(0.2974807540029168)  # 피치 각도 (45도)
yaw_angle = np.radians(-89.51503152538156)   # 요 각도 (60도)

# 회전 변환 행렬 생성
rotation_matrix = np.array([
    [np.cos(yaw_angle) * np.cos(pitch_angle), -np.sin(yaw_angle) * np.cos(roll_angle) + np.cos(yaw_angle) * np.sin(pitch_angle) * np.sin(roll_angle), np.sin(yaw_angle) * np.sin(roll_angle) + np.cos(yaw_angle) * np.sin(pitch_angle) * np.cos(roll_angle)],
    [np.sin(yaw_angle) * np.cos(pitch_angle), np.cos(yaw_angle) * np.cos(roll_angle) + np.sin(yaw_angle) * np.sin(pitch_angle) * np.sin(roll_angle), -np.cos(yaw_angle) * np.sin(roll_angle) + np.sin(yaw_angle) * np.sin(pitch_angle) * np.cos(roll_angle)],
    [-np.sin(pitch_angle), np.cos(pitch_angle) * np.sin(roll_angle), np.cos(pitch_angle) * np.cos(roll_angle)]
])

# 직육면체를 회전합니다.
cube.rotate(rotation_matrix, center=(0, 0, 0))

# 시각화
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(cube)
vis.run()
vis.destroy_window()

In [98]:
import open3d as o3d
import numpy as np
from ultralytics import YOLO
from crop_ply import *
import cv2

model = YOLO("modelFolder/model_v7.pt")
results = model(source="validimage/1_Color.png", device="cpu", project="results", name="test6d", conf=0.8)

# 2D 이미지의 네 꼭지점 좌표를 저장할 리스트
points_2D = []

for result in results:
    image_points = result.masks.xyn
    image_boxing = result.boxes.xywh.tolist()
    for box_info in image_boxing:
        x1, y1, w, h = box_info

        x2 = x1 + w
        y2 = y1
        x3 = x1
        y3 = y1 + h
        x4 = x1 + w
        y4 = y1 + h

        # 각 객체의 네 꼭지점 좌표를 2D 좌표로 추가
        points_2D.append([x1, y1, x2, y2, x3, y3, x4, y4])

print(points_2D)

xyz_output = cropPly(image_points, "validimage/1.ply")

def image_point_to_find(x, y):
    find_point = np.array([x, y], dtype=np.float32)
    closest_index = np.argmin(np.linalg.norm(find_point - np.array(cloud), axis=1))
    z_coordinate = xyz_output[closest_index, 2]  # 3D 좌표에서 z 좌표 추출
    return z_coordinate

for cloud in xyz_output:
     # 네 꼭짓점에 해당하는 2D 좌표를 정수로 변환
    x1, y1, x2, y2, x3, y3, x4, y4 = map(int, points_2D[0])

    obb = cloud.get_oriented_bounding_box()
    obb.color = (0, 1, 0)
    
    o3d.visualization.draw_geometries([cloud,obb])     
    
#2차원 영상좌표
points_2D = np.array([
                        (x1, y1),  #좌 하단 
                        (x2, y2),  #우 하단
                        (x3, y3),  #좌 상단
                        (x4, y4),  #우 상단
                      ], dtype="double")

#3차원 월드좌표
points_3D = np.array([
                      (point1_3d),       #좌 하단
                      (point2_3d),        #우 하단
                      (point3_3d),        #좌 상단
                      (point4_3d)          #우 상단
                     ], dtype="double")

print(points_3D)

# # 카메라 내부 파라미터
# focal_length = (959.094, 959.094)  # 초점 거리 (fx, fy)
# principal_point = (965.476, 521.368)  # 중심점 (cx, cy)

# # 카메라 매트릭스 생성
# camera_matrix = np.array([[focal_length[0], 0, principal_point[0]],
#                            [0, focal_length[1], principal_point[1]],
#                            [0, 0, 1]], dtype=np.float32)

# # 왜곡 계수 설정
# dist_coeffs = np.zeros((4, 1))

# # solvePnP 함수를 사용하여 포즈 추정
# retval, rvec, tvec = cv2.solvePnP(points_3D, points_2D, camera_matrix, dist_coeffs, rvec=None, tvec=None, useExtrinsicGuess=None, flags=None)

# R = cv2.Rodrigues(rvec)
# t = tvec

# print("Rotation Matrix:")
# print(R)
# print("\nTranslation Vector:")
# print(t)



image 1/1 c:\Team_Project-1\validimage\1_Color.png: 384x640 1 box, 960.5ms
Speed: 3.0ms preprocess, 960.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


[[605.517822265625, 401.0127868652344, 841.6450500488281, 401.0127868652344, 605.517822265625, 540.2096252441406, 841.6450500488281, 540.2096252441406]]
[[        605         401     -0.6792]
 [        841         401     -0.6792]
 [        605         540     -0.6792]
 [        841         540     -0.6792]]


In [73]:
import open3d as o3d
import numpy as np
from ultralytics import YOLO
from crop_ply import *
import cv2

# YOLO 모델 초기화
model = YOLO("modelFolder/model_v7.pt")

# 이미지에서 객체 검출
results = model(source="validimage/1_Color.png", device="cpu", project="results", name="test6d", conf=0.8)

# 2D 이미지의 네 꼭지점 좌표를 저장할 리스트
points_2D = []

for result in results:
    image_points = result.masks.xyn
    image_boxing = result.boxes.xywh.tolist()
    for box_info in image_boxing:
        x1, y1, w, h = box_info

        x2 = x1 + w
        y2 = y1
        x3 = x1
        y3 = y1 + h
        x4 = x1 + w
        y4 = y1 + h

        # 각 객체의 네 꼭지점 좌표를 2D 좌표로 추가
        points_2D.append([x1, y1, x2, y2, x3, y3, x4, y4])

# 포인트 클라우드 자르기 함수 호출
xyz_output = cropPly(image_points, "validimage/1.ply")

# 각 꼭짓점에 해당하는 포인트 클라우드의 Z 좌표 추출 및 출력
for cloud in xyz_output:
    # 네 꼭짓점에 해당하는 2D 좌표를 정수로 변환
    x1, y1, x2, y2, x3, y3, x4, y4 = map(int, points_2D[0])

    # 각 꼭짓점의 2D 좌표를 포인트 클라우드 좌표로 변환
    point1 = [x1, y1]
    point2 = [x2, y2]
    point3 = [x3, y3]
    point4 = [x4, y4]

    # Z 좌표를 추출
    z1 = cloud.get_max_bound()[2]  # 포인트 클라우드의 최대 Z 좌표
    z2 = cloud.get_max_bound()[2]
    z3 = cloud.get_max_bound()[2]
    z4 = cloud.get_max_bound()[2]

    # 각 꼭짓점의 3D 좌표를 구성
    point1_3d = [point1[0], point1[1], z1]
    point2_3d = [point2[0], point2[1], z2]
    point3_3d = [point3[0], point3[1], z3]
    point4_3d = [point4[0], point4[1], z4]

    # 각 꼭짓점의 3D 좌표 출력
    print("Point 1 (x, y, z):", point1_3d)
    print("Point 2 (x, y, z):", point2_3d)
    print("Point 3 (x, y, z):", point3_3d)
    print("Point 4 (x, y, z):", point4_3d)



image 1/1 c:\Team_Project-1\validimage\1_Color.png: 384x640 1 box, 898.2ms
Speed: 0.0ms preprocess, 898.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


Point 1 (x, y, z): [605, 401, -0.63623]
Point 2 (x, y, z): [841, 401, -0.63623]
Point 3 (x, y, z): [605, 540, -0.63623]
Point 4 (x, y, z): [841, 540, -0.63623]
